In [7]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim

import torchdiffeq

from tensorboard_utils import Tensorboard
from tensorboard_utils import tensorboard_event_accumulator

import transformer.Constants as Constants
from transformer.Layers import EncoderLayer, DecoderLayer
from transformer.Modules import ScaledDotProductAttention
from transformer.Models import Decoder, get_attn_key_pad_mask, get_non_pad_mask, get_sinusoid_encoding_table
from transformer.SubLayers import PositionwiseFeedForward

import dataset

import model_process
import checkpoints
from node_transformer import NodeTransformer

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook  
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2

Torch Version 1.1.0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [9]:
data = torch.load("/home/mandubian/datasets/multi30k/multi30k.atok.low.pt")

In [10]:
max_token_seq_len = data['settings'].max_token_seq_len
print(max_token_seq_len)

52


In [11]:
train_loader, val_loader = dataset.prepare_dataloaders(data, batch_size=128)

### Create an experiment with a name and a unique ID

In [12]:
exp_name = "transformer_6_layers_multi30k"
unique_id = "2019-06-07_1000"


### Create Model

In [13]:
model = None

In [14]:
src_vocab_sz = train_loader.dataset.src_vocab_size
print("src_vocab_sz", src_vocab_sz)
tgt_vocab_sz = train_loader.dataset.tgt_vocab_size
print("tgt_vocab_sz", tgt_vocab_sz)

if model:
    del model
    
model = NodeTransformer(
    n_src_vocab=max(src_vocab_sz, tgt_vocab_sz),
    n_tgt_vocab=max(src_vocab_sz, tgt_vocab_sz),
    len_max_seq=max_token_seq_len,
    n_layers=6,
    #emb_src_tgt_weight_sharing=False,
    #d_word_vec=128, d_model=128, d_inner=512,
    n_head=8, method='dopri5-ext', rtol=1e-3, atol=1e-3,
    has_node_encoder=False, has_node_decoder=False)

model = model.to(device)

src_vocab_sz 9795
tgt_vocab_sz 17989


### Create Tensorboard metrics logger

In [15]:
tb = Tensorboard(exp_name, unique_name=unique_id)

Writing TensorBoard events locally to runs/transformer_6_layers_multi30k_2019-06-07_1000


### Create basic optimizer

In [16]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.995), eps=1e-9)


### Train

In [17]:
# Continuous space discretization
timesteps = np.linspace(0., 1, num=6)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 50
LOG_INTERVAL = 5

#from torch import autograd
#with autograd.detect_anomaly():
model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    #start_epoch=0, best_valid_accu=state["acc"]
)

Loaded model and timesteps to cuda


[ Epoch 0 ]
Adding group train to writers (dict_keys([]))
[Training]  loss: 7.2235593511588565, ppl:  1371.36154, accuracy: 16.599 %, elapse: 62618.487ms
Adding group eval to writers (dict_keys(['train']))
[Validation]  loss: 5.975024863551293,  ppl:  393.47788, accuracy: 27.932 %, elapse: 898.344ms
Checkpointing Validation Model...


[ Epoch 1 ]
[Training]  loss: 6.131624574610461, ppl:  460.18316, accuracy: 30.963 %, elapse: 62870.110ms
[Validation]  loss: 5.2839826120245,  ppl:  197.15350, accuracy: 33.047 %, elapse: 898.867ms
Checkpointing Validation Model...


[ Epoch 2 ]
[Training]  loss: 5.611737301748947, ppl:  273.61918, accuracy: 35.105 %, elapse: 62964.462ms
[Validation]  loss: 4.799503687417168,  ppl:  121.45013, accuracy: 36.256 %, elapse: 898.765ms
Checkpointing Validation Model...


[ Epoch 3 ]
[Training]  loss: 5.249176728696815, ppl:  190.40945, accuracy: 38.443 %, elapse: 63148.525ms
[Validation]  loss: 4.434336158571531,  ppl:  84.29615, accuracy: 39.802 %, elapse: 900.744ms
Checkpointing Validation Model...


...

[ Epoch 45 ]
[Training]  loss: 2.216281681360717, ppl:  9.17316, accuracy: 82.921 %, elapse: 63131.145ms
[Validation]  loss: 3.17959511868463,  ppl:  24.03702, accuracy: 54.875 %, elapse: 900.291ms


[ Epoch 46 ]
[Training]  loss: 2.185835235953352, ppl:  8.89808, accuracy: 83.551 %, elapse: 62791.074ms
[Validation]  loss: 3.1733583130171397,  ppl:  23.88757, accuracy: 55.298 %, elapse: 903.101ms


[ Epoch 47 ]
[Training]  loss: 2.1569908027534717, ppl:  8.64508, accuracy: 84.064 %, elapse: 62907.164ms
[Validation]  loss: 3.2005433952221685,  ppl:  24.54586, accuracy: 55.333 %, elapse: 900.654ms
Checkpointing Validation Model...


[ Epoch 48 ]
[Training]  loss: 2.127601170555993, ppl:  8.39471, accuracy: 84.720 %, elapse: 62784.837ms
[Validation]  loss: 3.1910431609982806,  ppl:  24.31378, accuracy: 55.040 %, elapse: 899.151ms


[ Epoch 49 ]
[Training]  loss: 2.1024699563810847, ppl:  8.18636, accuracy: 85.206 %, elapse: 62871.950ms
[Validation]  loss: 3.211829971367039,  ppl:  24.82447, accuracy: 54.939 %, elapse: 913.650ms



### Restore best checkpoint (to restart past training)

In [ ]:
state = checkpoints.restore_best_checkpoint(
    "node_transformer_multi30k", "2019-05-29_1200", "validation", model, optimizer)

print("accuracy", state["acc"])
print("loss", state["loss"])
model = model.to(device)